In [1]:
# Librairies
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import random
from pprint import pprint
import time
from joblib import Parallel, delayed, parallel_backend, Memory
import math

In [20]:
#Differents functions
# spliting datas
def train_test_split(data, test_size):
    
    #check if test_size is a float
    if isinstance(test_size, float):
        test_size = round(test_size * len(data))
    
    #retrieve all indices
    indices = data.index.tolist()
    
    #select random indices according to test_size
    test_indices = random.sample(population=indices, k=test_size)
    
    #spliting datas in training & testing
    test_data = data.loc[test_indices]
    train_data = data.drop(test_indices)
    
    return train_data, test_data


#Check data purity
def check_purity(data):
    #Return purity value regarding to the number of unique classes
    return  (False,True)[len(np.unique(data[:,-1])) == 1]


#Normal classification using majority
def classify(data):
    #get all classes & count occurences
    label = data[:,-1]
    uniques_classes, counts = np.unique(data[:,-1], return_counts = True)
    
    #get index of most common class
    index = counts.argmax()
    
    #return the most common class
    return uniques_classes[index]


# Modified Classification using proposed approach
def modified_classify(data):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = 0
    index_taux = 4
    index_nbe = 3
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant]
        #print("({} , {})".format(data[line,index_montant], data[line, index_taux]))
    
    #print("({}, {})".format(P, M))
    
    return (0, 1)[M > P]
        

#Calculate all potential splits
def get_splits(data):
    
    #initialize our dict of potential splits
    splits = {}
    
    #initialize all attribute potential splits list
    _, n_cols = data.shape
    for col in range(n_cols - 1):
        if col == 0:
            continue
        splits[col] = list()
    
    # Compute all attributes potential splits
    for col in range(n_cols - 1):
        if col == 0:
            continue
        #get unique datas
        #print("Current col: {}".format(col))
        values = np.unique(data[:, col])
        #populate our dict
        feature_type = FEATURE_TYPES[col]
        if feature_type == "Continous":
            for index in range(1, len(values)):
                current_value = values[index]
                #print(current_value)
                previous_value = values[index - 1]
                potential_split = np.mean([current_value, previous_value])
                splits[col].append(potential_split)
        else:
            splits[col] = values
    return splits


#Spliting data
def split_data(data, feature_col, value):
    
    feature_type = FEATURE_TYPES[feature_col]
    
    #define all masks
    
    if feature_type == "Continous":
        mask_inf = data[:, feature_col] <= value
        mask_sup = data[:, feature_col] > value
    else:
        mask_inf = data[:, feature_col] == value
        mask_sup = data[:, feature_col] != value
    
    #data spliting
    data_inf = data[mask_inf]
    data_sup = data[mask_sup]
    
    return data_inf, data_sup


#Calculate chosen metric
def calculate_metric(data):
     #get classes
    label_class = data[:, -1]
    #get counts for each class
    _, counts = np.unique(label_class, return_counts=True)
    
    probabilities = counts / counts.sum()
    
    #computing metric value depending on the user choice
    if METRIC == "entropy":
        probabilities = counts / counts.sum()
        computed_metric = sum(probabilities * -np.log2(probabilities))
    elif METRIC == "gini":
        probabilities **=2
        computed_metric = 1- sum(probabilities)
    
    return computed_metric


# Overall metric value
def overall_metric(data_inf, data_sup):
    #get number of datas
    data_all_lenght = len(data_inf) + len(data_sup)
    
    #compute overall metric value
    metric_data_inf = (len(data_inf) / data_all_lenght)*calculate_metric(data_inf)
    metric_data_sup = (len(data_sup) / data_all_lenght)*calculate_metric(data_sup)
    overall_metric= metric_data_inf + metric_data_sup
    
    return overall_metric


#Modified metric: Here the aim is to compute the total lost that we'll have
def modified_metric(data):
    #initializing
    P  = 0
    M = 0
    n_row , _ = data.shape
    index_montant = 0
    index_taux = 4
    index_nbe = 3
    for line in range(n_row):
        if data[line,-1] == 0:
            P += data[line,index_montant]
        else:
            M += (data[line, index_taux]/100) * data[line,index_montant]
    #print(P)
    return P


#Compute our overall modified metric
def overall_modified_metric(data_inf, data_sup):
    #get number of datas
    data_all_lenght = len(data_inf) + len(data_sup)
    
    #compute overall metric value
    metric_data_inf = (len(data_inf) / data_all_lenght)*modified_metric(data_inf)
    metric_data_sup = (len(data_sup) / data_all_lenght)*modified_metric(data_sup)
    overall_metric= metric_data_inf + metric_data_sup
    #print("({}, {})".format(metric_data_inf, metric_data_sup))
    
    return overall_metric


#parralelized function
def computing_best_column(data, colum_index, value):
    
    global best_split_column, best_split_value, overall_metric_value
    
    #print("current column: {}".format(colum_index))
    data_inf, data_sup = split_data(data, colum_index, value)
    current_overall = overall_metric(data_inf, data_sup)
            #print(current_overall)
            #check if lower
    if current_overall <= overall_metric_value:
        print("cur,met, val, col, : ({}, {},{},{})".format(current_overall, overall_metric_value, colum_index, value))
        overall_metric_value = current_overall
        best_split_column = colum_index
        best_split_value = value
        #print("Done !! bests: ({}, {}, {})".format(best_split_column, best_split_value, overall_metric_value))

        
#determine best split attribute and value
def determine_best_split(data, potential_splits):
     
    #print(potential_splits)
    
    #So let's implement parralel version of our super code
    #Parallel(n_jobs=-1, require='sharedmem')(
     #   delayed(func)(args)
    #)
    global best_split_column, best_split_value, overall_metric_value
    
    overall_metric_value = 300000000000000
    
    with Parallel(n_jobs=3, backend="threading", require="sharedmem", verbose=5) as parallel:
        parallel(delayed(computing_best_column)(data, colum_index, value) for colum_index in potential_splits for value in potential_splits[colum_index]) 
    
    """
        for colum_index in potential_splits:
        print("current column: {}".format(colum_index))
        for value in potential_splits[colum_index]:
            print("current value: {}".format(value))
            data_inf, data_sup = split_data(data, colum_index, value)
            current_overall = overall_metric(data_inf, data_sup)
            #print(current_overall)
            #check if lower
            if current_overall <= overall_metric_value:
                overall_metric_value = current_overall
                best_split_column = colum_index
                best_split_value = value
    #print(best_split_value)
    """
    #Loop over all datas, calculate overall_entropy, and update if it's lower
    
    print("Final Done !! bests: ({}, {}, {})".format(best_split_column, best_split_value, overall_metric_value))
    return best_split_column, best_split_value, overall_metric_value


#building decision Tree
def decision_tree(df, counter=0, min_samples=5, max_depth=5, metric="entropy"):
    
    if counter == 0:
        data = df.values
        global COLUMNS_NAMES, FEATURE_TYPES, METRIC
        COLUMNS_NAMES = df.columns[:-1]
        FEATURE_TYPES = determine_feature_types(df)
        METRIC = metric
    else:
        data = df
        
    #base case
    
    if check_purity(data) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify(data)
        return classification
    
    else:
        counter +=1
        #computations for right and left part
        potential_splits = get_splits(data)
        best_split_column, best_split_value, overall_metric_value = determine_best_split(data, potential_splits)
        #We must change data_inf & data_sup order later
        data_inf, data_sup = split_data(data, best_split_column, best_split_value)
        
        # Creating subTree
        feature_type = FEATURE_TYPES[best_split_column]    
        if feature_type == "Continous":
            question = "{} <= {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        else:
            question = "{} == {}".format(COLUMNS_NAMES[best_split_column], best_split_value)
        #Adding labels
        labels = " " + str(len(data_inf) + len(data_sup)) + " " + str(overall_metric_value)
        question += labels
        sub_tree = {question: []}
        print(question)
            
        
        #left and right
        yes_answer = decision_tree(data_inf, counter, min_samples, max_depth, metric=METRIC)
        no_answer = decision_tree(data_sup, counter, min_samples, max_depth, metric=METRIC)
        
        if yes_answer == no_answer:
            print(yes_answer)
            sub_tree = yes_answer
        else:
            #Append left and right part
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

    
#classify a sample
def classify_sample(sample, tree):
    
    #get node elements
    if not isinstance(tree, dict):
        return tree
    key = list(tree.keys())[0]
    #print(key.split())
    feature, comp_op, value, _, _ = key.split()
    
    if comp_op == "<=":
        if sample[feature] <= float(value):
            answer = tree[key][0]
        else:
            answer = tree[key][1]
    else:
        if str(sample[feature]) == value:
            answer = tree[key][0]
        else:
            answer = tree[key][1]
        
    #test base case
    if not isinstance(answer, dict):
        #print('yes')
        return answer
    else:
        return classify_sample(sample, answer)

    
#compute accuracy
def my_accuracy(df, tree):
    
    df["classification"] = df.apply(classify_sample, axis=1, args=(tree,))
    df["classification_correct"] = df.classification == df.loan_status
    
    accuracy = df.classification_correct.mean()
    
    return accuracy, df
    

# In order to handle non-continous values
# We have to identify all features type in our dataset
def determine_feature_types(data):
    
    features_type  = []
    threshold = 931
    for col in data.columns:
        
        uniques_val = data[col].unique()
        sample = uniques_val[0]
        
        if (isinstance(sample, str)):
            features_type.append("Categorical")
        else:
            features_type.append("Continous")
    
    return features_type

In [ ]:
#Execution

#random.seed(30)
#datas = pd.read_csv('datas/SouthGermanCredit.asc', sep=' ')

datas = pd.read_excel('datas_loan.xlsx', index=False)
datas.emp_length = datas.emp_length.astype(str)
datas = datas.drop(columns=['Unnamed: 0'])

train, test = train_test_split(datas, 0.3)
start = time.time()
my_tree = decision_tree(train, min_samples=60, max_depth=5, metric="gini")
acc, df = my_accuracy(test, my_tree)
end = time.time()

In [32]:
#Computing new matrix confusion values
P = df.loan_amount[(df.loan_status == 0) & (df.classification == 1)].sum()
R = df.loan_amount[(df.loan_status == 1) & (df.classification == 0)].sum()

df_no_win = df[["loan_amount", "interest_rate", "term"]][(df.loan_status == 1) & (df.classification == 0)]

df_no_win['interest_rate'] /=100.0
df_no_win["NoWin"] = df_no_win.loan_amount * df_no_win.interest_rate * df_no_win.term/12

M = df_no_win.NoWin.sum()
P, M, R

(11949000, 8104.38, 20200)

In [ ]:
TP = len(df[(df.loan_default == 1) & (df.classification == 1)])
FP = len(df[(df.loan_default == 0) & (df.classification == 1)])
FN = len(df[(df.loan_default == 1) & (df.classification == 0)])

Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
Precision, Recall